In [1]:
%cd ./YOLOP

C:\Users\Nathan\Desktop\Test Detection\lane-detection\YOLOP


## Parameters

In [12]:
################################
# Directories
################################

# directory where videos are being performed detections on
source_dir = "../videos"

# folder for where images are saved if detection performed on images
save_dir = "../save"

# folder where raw output from detections are saved as csv
csv_dir = "../csv_raw_output"

# folder where post processed csvs are being saved
processed_csv_dir = "../processed"

# folder where external bounding boxes are being used
external_bounding_boxes = "../external_bounding_boxes"

# folder where pandas friendly csv are being saved
zeros_dir = "../lane_zeros"

# folder for storing only the closest bounding boxes (-1 0 1)
closest_dir = "../closest_boxes"


################################
# Settings for DETECTION ONLY
################################

# set a center so it doesn't need to check for a new center
# there's a cell at the bottom that can help find the center
preset_center = 615

# fps for detection to run at
detection_fps = 30

# minimum acceptable confidence 
confidence_threshold = 0.5

# Will not perform detection unless there are external bounding boxes for that video
enforce_external_boxes = False

list of dictionaries with keys ["frame", "rois", "class_ids", "scores"]

##Functions

In [4]:
import os
import csv
import statistics as stats
import cv2

def read_csv_detection(detection):
  detection_data = detection.split(" ")
  if detection_data[0] == "None":
    lane_number = None
  else:
    lane_number = int(detection_data[0])
  x1 = int(float(detection_data[1]))
  y1 = int(float(detection_data[2]))
  x2 = int(float(detection_data[3]))
  y2 = int(float(detection_data[4]))
  conf = float(detection_data[5])
  frame_center = int(float(detection_data[6]))
  if len(detection_data) >= 8:
    vehicle_class = int(float(detection_data[7]))
    return lane_number, x1, y1, x2, y2, conf, frame_center, vehicle_class
  return lane_number, x1, y1, x2, y2, conf, frame_center

def find_lane_zero(detections):
  for detection in detections:
    data = read_csv_detection(detection)
    if data[0] == 0:
      return data
  return None

#box = [x1,y1,x2,y2]
#returns mean and standard deviations list of each coordinate
def get_boxes_stats(boxes):
  stdev = None
  mean = None
  if len(boxes) > 0:
    x1_list = [x1[0] for x1 in boxes]
    y1_list = [y1[1] for y1 in boxes]
    x2_list = [x2[2] for x2 in boxes]
    y2_list = [y2[3] for y2 in boxes]
    mean = [int(stats.mean(x1_list)), int(stats.mean(y1_list)), int(stats.mean(x2_list)), int(stats.mean(y2_list))]
    stdev = [stats.pstdev(x1_list), stats.pstdev(y1_list), stats.pstdev(x2_list), stats.pstdev(y2_list)]
  return mean, stats.mean(stdev)

def find_largest_box(detections, target_lane_number):
  largest_detection = None
  for detection in detections:
    lane_number, x1, y1, x2, y2 = read_csv_detection(detection)[:5]
    if(target_lane_number == lane_number):
      if(largest_detection is None):
        largest_detection = detection
      else:
        _ , l_x1, l_y1, l_x2, l_y2 = read_csv_detection(largest_detection)[:5]
        largest_area = (l_x2 - l_x1) * (l_y2 - l_y1)
        area = (x2-x1) * (y2-y1)
        if area > largest_area:
          largest_detection = detection
  return largest_detection

## Detection

In [14]:
cmd_source_dir = "\"" + source_dir + "\""
cmd_save_dir = "\"" + save_dir + "\""
cmd_csv_dir = "\"" + csv_dir + "\""

cmd_external_boxes = None
if external_bounding_boxes:
  cmd_external_boxes = "\"" + external_bounding_boxes + "\""

if preset_center:
  !python tools/demo.py --source {cmd_source_dir} --save-dir {cmd_save_dir}  --detect-fps {detection_fps} --save-csv {cmd_csv_dir} --external-boxes {cmd_external_boxes} --img-center {preset_center} --enforce-external {enforce_external_boxes}
else:
  !python tools/demo.py --source {cmd_source_dir} --save-dir {cmd_save_dir}  --detect-fps {detection_fps} --save-csv {cmd_csv_dir} --external-boxes {cmd_external_boxes} --enforce-external {enforce_external_boxes}

^C


## Post-Processing
detection format: [lane number, x1, y1, x2, y2, confidence, lane center]

In [15]:
import os
import csv
import statistics as stats
import cv2

for detection_csv in os.listdir(csv_dir):
  detection_csv_path = os.path.join(csv_dir, detection_csv)
  
  video_name = detection_csv.split(".csv")[0]

  if video_name in os.listdir(source_dir):
    source_video_path = os.path.join(source_dir, video_name)
    video_reader = cv2.VideoCapture(source_video_path)
    video_res = [int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))]
    video_reader.release()
    print("using source video")
  else:
    continue


  # adds detections grouped by frame
  # dictionary {frame_number: [detections]}
  detections_by_frame = {}
  print(video_name)
  with open(detection_csv_path, "r") as detection_csv_file:
    csv_reader = csv.reader(detection_csv_file)
    for row in list(csv_reader):
      frame_number = int(float(row[0]))
      detections_by_frame[frame_number] = row[1:]

  # FIRST PASS
  # decides lane 0 cars
  for frame in sorted(detections_by_frame.keys()):
    # defines lane 0 through area and distance from center
    minimum_diff = None
    for detection in detections_by_frame[frame]:
      if float(detection.split(" ")[5]) < confidence_threshold: # will skip detections below confidence threshold
        continue
      lane_number, x1, y1, x2, y2, conf, frame_center = read_csv_detection(detection)[:7]
      if lane_number == 0: # chooses lane zero through weighted parameters
        midpoint = int(stats.mean([x1, x2]))
        diff = abs(frame_center - midpoint)
        bound_box_area = (x2 - x1) * (y2 - y1)
        img_area = video_res[0] * video_res[1]
        area_weight = 1 - ((bound_box_area / img_area) ** 0.2)
        diff_weight = (diff / video_res[0]) ** 2 + 0.075
        weight = diff_weight * area_weight
        if minimum_diff is None or weight < minimum_diff[1]:
          minimum_diff = [[x1,y1,x2,y2], weight]

    # removes all other lane 0 except for one car
    for i in range(len(detections_by_frame[frame])):
      detection = detections_by_frame[frame][i]

      lane_number, x1, y1, x2, y2, conf, frame_center = read_csv_detection(detection)[:7]

      if(len(detection.split(" ")) >= 8):
        class_id = read_csv_detection(detection)[7]

      if lane_number == 0: # remove all other lane 0
        if not minimum_diff is None:
          if [x1,y1,x2,y2] != minimum_diff[0]:
            midpoint = int(stats.mean([x1, x2]))
            new_lane_number = 0
            if midpoint > frame_center:
              new_lane_number = 1
            else:
              new_lane_number = -1
            # adjust lane number of false lane 0
            detections_by_frame[frame][i] = f"{new_lane_number} {x1} {y1} {x2} {y2} {conf} {frame_center}"
            if(len(detection.split(" ")) >= 8):
              detections_by_frame[frame][i] += f" {class_id}"
            # end adjusting
  # END FIRST PASS


  # SECOND PASS
  # stabilizes lane 0 cars from the past
  previous_frame_count = 5
  for frame_index in range(len(detections_by_frame.keys())):
    if frame_index != 0 and frame_index != len(detections_by_frame) - 1:

      # gathers all the lane 0 bounding boxes from the past 5 frames
      prev_centers = []
      for i in range(previous_frame_count):
        if frame_index - i < 0:
          continue
        detection_data = find_lane_zero(detections_by_frame[list(detections_by_frame.keys())[frame_index - 1]])
        if not detection_data is None:
          lane_number, x1, y1, x2, y2, conf, frame_center = detection_data[:7]

          center_previous = (stats.mean([x1, x2]), stats.mean([y1, y2]))
          prev_centers.append(center_previous)
        else:
          continue
      # end gathering

      #finds the minimum difference between current lane 0 and past 5 lane 0 cars
      #sets the default minimum difference to compare to (current lane 0)
      minimum_diff = []
      detection_data = find_lane_zero(detections_by_frame[list(detections_by_frame.keys())[frame_index]])
      if not detection_data is None:
        lane_number, x1, y1, x2, y2, conf, frame_center = detection_data[:7]

        prev_diffs = []
        center_zero = (stats.mean([x1, x2]), stats.mean([y1, y2]))
        for center_previous in prev_centers:
          prev_diff = abs(center_zero[0] - center_previous[0]) + abs(center_zero[1] - center_previous[1])
          prev_diffs.append(prev_diff)
        if len(prev_diffs) == 0:
          continue

        original = [(x1,y1,x2,y2), stats.mean(prev_diffs)]
        minimum_diff = [(x1,y1,x2,y2), stats.mean(prev_diffs)]

      else:
        continue
      # end finding min difference of lane 0

      #finds minimum difference between all bounding boxes and past 5 lane 0 cars
      for i in range(len(detections_by_frame[list(detections_by_frame.keys())[frame_index]])):
        detection = detections_by_frame[list(detections_by_frame.keys())[frame_index]][i]
        detection_data = read_csv_detection(detection)

        lane_number, x1, y1, x2, y2, conf, frame_center = detection_data[:7]

        if len(detection_data) <= 7:
          continue

        # list of differences between current box and past boxes
        prev_diffs = []
        center_curr = (stats.mean([x1, x2]), stats.mean([y1, y2]))
        for prev_center in prev_centers:
          prev_diff = abs(center_curr[0] - prev_center[0]) + abs(center_curr[1] - prev_center[1])
          prev_diffs.append(prev_diff)
        if len(prev_diffs) == 0:
          continue
        
        if stats.mean(prev_diffs) < minimum_diff[1]:
          minimum_diff = [(x1,y1,x2,y2), stats.mean(prev_diffs), i]
      # end finding min difference of all bounding boxes


      # assigns new lane 0 car with minimum difference, and gives lane number to old lane 0 car
      if minimum_diff[0] != original[0] and abs(minimum_diff[1] - original[1]) > 0:
        if len(minimum_diff) > 2:
          # removes old lane 0 car
          new_zero_detection = detections_by_frame[list(detections_by_frame.keys())[frame_index]][minimum_diff[2]]
          old_zero = find_lane_zero(detections_by_frame[list(detections_by_frame.keys())[frame_index]])

          for i in range(len(detections_by_frame[list(detections_by_frame.keys())[frame_index]])):
            if detections_by_frame[list(detections_by_frame.keys())[frame_index]][i] == old_zero:
              lane_number, x1, y1, x2, y2, conf, frame_center = old_zero
              if (len(old_zero) >= 8):
                class_id = old_zero[7]

            # adjust lane number of false lane 0
            new_lane_number = 0
            if stats.mean([x1,x2]) > frame_center:
              new_lane_number = 1
            else:
              new_lane_number = -1
            write_detection = f"{new_lane_number} {x1} {y1} {x2} {y2} {conf} {frame_center}"
            
            if enforce_external_boxes:
              write_detection += f" {class_id}"
            detections_by_frame[list(detections_by_frame.keys())[frame_index]][i] = write_detection
            # end adjustment

          # write the new lane 0 car
          detection_data = read_csv_detection(new_zero_detection)
          lane_number, x1, y1, x2, y2, conf, frame_center = detection_data[:7]

          write_detection = f"0 {x1} {y1} {x2} {y2} {conf} {frame_center}"
          if(len(detection_data) >= 8):
            write_detection += f" {detection_data[7]}"
          # end writing new lane 0 car
            
          detections_by_frame[list(detections_by_frame.keys())[frame_index]][minimum_diff[2]] = write_detection
      # end assignment
  # END SECOND PASS



  # THIRD PASS 
  # detects anomoly
  # this pass repeats to adjust all anomolies (gaps or jumps)
  max_range = 7
  minimum_list_size = 3
  max_st_dev = 5
  mean_difference_threshold = 75
  max_passes = 5
  pass_number = 0

  anomoly_detected = True
  while pass_number < max_passes and anomoly_detected is True:
    anomoly_detected = False
    for frame_index in range(len(detections_by_frame.keys())):
      
      # determine if there is a gap or jump
      current_detection = find_lane_zero(detections_by_frame[list(detections_by_frame.keys())[frame_index]])

      # gather lane 0 bounding boxes before current detection
      past_boxes = []
      for i in range(max_range):
        if frame_index - 1 - i >= 0:
          past_box = find_lane_zero(detections_by_frame[list(detections_by_frame.keys())[frame_index - 1 - i]])
          if not past_box is None:
            past_boxes.append(past_box[1:5])
            if(len(past_box) >= 8):
              predicted_class_id = past_box[7]
            
      # gather lane 0 bounding boxes after current detection
      future_boxes = []
      for i in range(max_range):
        if frame_index + 1 + i < len(detections_by_frame.keys()):
          future_box = find_lane_zero(detections_by_frame[list(detections_by_frame.keys())[frame_index + 1 + i]])
          if not future_box is None:
            future_boxes.append(future_box[1:5])

      # remove outliers
      # outliers are detections with lack of good detections near it
      past_coord_mean, past_st_dev = None, None
      if len(past_boxes) != 0:
        past_coord_mean, past_st_dev = get_boxes_stats(past_boxes)

      future_coord_mean, future_st_dev = None, None
      if len(future_boxes) != 0:
        future_coord_mean, future_st_dev = get_boxes_stats(future_boxes)
      
      past_detections_good = True
      if len(past_boxes) < minimum_list_size or past_st_dev > max_st_dev:
        past_detections_good = False

      future_detections_good = True
      if len(future_boxes) < minimum_list_size or future_st_dev > max_st_dev:
        future_detections_good = False

      # missing 1 side or both sides
      if not past_detections_good or not future_detections_good:
        if not current_detection is None:
          remove_lane_zero = False

          # complete lack of data
          if not past_detections_good and not future_detections_good:
            removal_reason = "due to complete lack of good data"
            remove_lane_zero = True
          
          # 1 sided data that doesn't match
          else:
            if len(past_boxes) >= minimum_list_size:
              other_coord_mean = past_coord_mean
              removal_reason = "due to past boxes lacking data"
            else:
              other_coord_mean = future_coord_mean
              removal_reason = "due to future boxes lacking data"
            current_coords = current_detection[1:5]
            # check mean difference from current coord to one side (past or present)
            if stats.mean([abs(current_coords[i] - other_coord_mean[i]) for i in range(len(current_coords))]) > mean_difference_threshold:
              remove_lane_zero = True

          # remove lane 0 detection
          if remove_lane_zero:
            anomoly_detected = True
            print("outlier", list(detections_by_frame.keys())[frame_index], past_st_dev, future_st_dev, removal_reason)
            for i in range(len(detections_by_frame[list(detections_by_frame.keys())[frame_index]])):
              detection = read_csv_detection(detections_by_frame[list(detections_by_frame.keys())[frame_index]][i])

              lane_number, x1, y1, x2, y2, conf, frame_center = detection[:7]

              new_lane_number = 0
              if stats.mean([x1,x2]) > frame_center:
                new_lane_number = 1
              else:
                new_lane_number = -1
              write_detection = f"{new_lane_number} {x1} {y1} {x2} {y2} {conf} {frame_center}"
              if len(detection) >= 8:
                write_detection += f" {detection[7]}"
              detections_by_frame[list(detections_by_frame.keys())[frame_index]][i] = write_detection
        continue
      
      average_difference = None

      predicted_current_coords = get_boxes_stats([past_coord_mean, future_coord_mean])[0]



      # get average differences
      if not current_detection is None:
        current_coords = current_detection[1:5]
        average_difference = stats.mean([abs(current_coords[i] - predicted_current_coords[i]) for i in range(len(current_coords))])

      # either gap or jump here
      if current_detection is None or average_difference > mean_difference_threshold:
        anomoly_detected = True
        print("anomoly detected", list(detections_by_frame.keys())[frame_index], average_difference, predicted_current_coords)
        
        # removes old detection
        if not current_detection is None:
          for i in range(len(detections_by_frame[list(detections_by_frame.keys())[frame_index]])):
            detection = read_csv_detection(detections_by_frame[list(detections_by_frame.keys())[frame_index]][i])
            lane_number, x1, y1, x2, y2, conf, frame_center = detection[:7]
                      
            if stats.mean([x1,x2]) > frame_center:
              new_lane_number = 1
            else:
              new_lane_number = -1

            # adjusts lane number of false lane 0
            write_detection  = f"{new_lane_number} {x1} {y1} {x2} {y2} {conf} {frame_center}"
            if len(detection) >= 8:
              write_detection += f" {detection[7]}"
            detections_by_frame[list(detections_by_frame.keys())[frame_index]][i] = write_detection
            # end adjust

        # create lane 0 prediction from averages
        x1, y1, x2, y2 = predicted_current_coords
        conf = 1
        frame_center = int(float(detections_by_frame[list(detections_by_frame.keys())[frame_index]][0].split(" ")[6]))

        # write lane 0 detection to fill gap
        write_detection = f"0 {x1} {y1} {x2} {y2} {conf} {frame_center}"
        if enforce_external_boxes:
          write_detection += f" {predicted_class_id}"
        detections_by_frame[list(detections_by_frame.keys())[frame_index]].append(write_detection)
        # end write lane 0 detection

    pass_number += 1
  # END THIRD PASS
      
  # creates directory
  if not (os.path.exists(processed_csv_dir) and os.path.isdir(processed_csv_dir)):
    os.mkdir(processed_csv_dir)
  processed_csv_path = os.path.join(processed_csv_dir, detection_csv)

  # writes to csv
  with open(processed_csv_path, "w", newline='') as detection_csv_file:
    csv_writer = csv.writer(detection_csv_file)
    for frame in sorted(detections_by_frame.keys()):
      detection_length = 0
      if enforce_external_boxes:
        detection_length = 8
      else:
        detection_length = 7
      csv_writer.writerow([frame] + [" ".join(x.split(" ")[:detection_length]) for x in detections_by_frame[frame]])


  # below handles writing to lane zeros
  detection_csv_path = os.path.join(csv_dir, detection_csv)
  video_name = detection_csv.split(".csv")[0]

  video_frames = video_res[2]
  print(video_frames)

  # create header and empty csv file
  lane_zeros = []
  lane_zeros.append(["Frame number", "x1", "y1", "x2", "y2", "confidence", "Lane center", "Class"])
  for i in range(video_frames + 1):
    lane_zeros.append([i + 1, "NA", "NA", "NA", "NA", "NA", "NA", "NA"])

  # gather lane zeros
  for frame in sorted(detections_by_frame.keys()):
    lane_zero = find_lane_zero(detections_by_frame[frame])
    if not lane_zero is None:
      lane_number, x1, y1, x2, y2, conf, frame_center = lane_zero[0:7]

      lane_zeros[frame] = [frame, x1, y1, x2, y2, conf, frame_center]
      if len(lane_zero) >= 8:
        lane_zeros[frame].append(lane_zero[7]) # class_id
      #print(frame)
        
  # create folder
  if not (os.path.exists(zeros_dir) and os.path.isdir(zeros_dir)):
    os.mkdir(zeros_dir)

  # write to csv file
  zeros_csv_dir = os.path.join(zeros_dir, detection_csv)
  with open(zeros_csv_dir, 'w', newline='') as zeros_csv:
    csv_writer = csv.writer(zeros_csv)
    csv_writer.writerows(lane_zeros)


using source video
20200306_123926_EF.mp4
outlier 5 5.172355213109037 5.62062719799557 due to complete lack of good data
outlier 6 5.172355213109037 5.6109302692547605 due to complete lack of good data
outlier 7 5.172355213109037 5.743853709874486 due to complete lack of good data
outlier 8 5.172355213109037 5.832420415576075 due to complete lack of good data
outlier 12 5.229974135373411 6.029472059684352 due to complete lack of good data
outlier 13 5.229974135373411 6.133081536226731 due to complete lack of good data
outlier 14 5.229974135373411 6.056943178539585 due to complete lack of good data
outlier 15 5.229974135373411 5.879508237420512 due to complete lack of good data
outlier 16 5.229974135373411 5.76430644409956 due to complete lack of good data
outlier 17 5.375 5.53476656992194 due to complete lack of good data
outlier 18 0.0 5.033357867014768 due to complete lack of good data
outlier 40 5.161554445513393 5.456156794350426 due to complete lack of good data
outlier 41 5.38249

anomoly detected 459 None [628, 448, 645, 462]
anomoly detected 460 None [628, 448, 645, 462]
anomoly detected 463 None [627, 448, 645, 462]
anomoly detected 464 None [627, 448, 645, 462]
anomoly detected 473 None [626, 446, 644, 461]
outlier 512 7.172507461619134 7.4006802115199255 due to complete lack of good data
outlier 513 7.644216800789881 9.588315897723707 due to complete lack of good data
outlier 514 8.16167368140847 9.588315897723707 due to complete lack of good data
outlier 515 8.83456490391585 10.70317454781796 due to complete lack of good data
outlier 516 9.614598750094682 10.679298019725726 due to complete lack of good data
outlier 517 10.125 10.63098108952136 due to complete lack of good data
outlier 518 0.0 9.839648393846112 due to complete lack of good data
outlier 519 None 9.765188967572424 due to complete lack of good data
outlier 520 None 7.710338447998057 due to complete lack of good data
outlier 521 None 7.688850750697299 due to complete lack of good data
outlier 8

anomoly detected 1169 87.25 [594, 439, 662, 501]
anomoly detected 1189 None [590, 439, 660, 503]
anomoly detected 1251 152 [588, 438, 659, 502]
anomoly detected 1923 None [581, 435, 669, 515]
anomoly detected 1924 None [581, 435, 669, 515]
outlier 42 5.587053685776047 5.2827956959925295 due to complete lack of good data
outlier 45 5.416741485536182 0.25 due to complete lack of good data
outlier 46 5.0962151943920775 0.0 due to complete lack of good data
outlier 51 0.0 5.364381276368433 due to complete lack of good data
outlier 56 None 5.28919784941919 due to complete lack of good data
outlier 57 None 5.717583095306143 due to complete lack of good data
outlier 58 None 6.125 due to complete lack of good data
outlier 59 None 0.0 due to complete lack of good data
outlier 60 None None due to complete lack of good data
outlier 195 8.013876853447538 0.0 due to complete lack of good data
outlier 196 8.5 None due to complete lack of good data
outlier 241 5.974830277570721 5.99207016715734 due t


1929it [25:24,  1.26it/s]
1930it [25:25,  1.24it/s]
1931it [25:26,  1.20it/s]
1932it [25:26,  1.19it/s]
1933it [25:27,  1.18it/s]
1934it [25:28,  1.20it/s]
1935it [25:29,  1.20it/s]
1936it [25:30,  1.20it/s]
1937it [25:31,  1.23it/s]
1938it [25:31,  1.23it/s]
1939it [25:32,  1.21it/s]
1940it [25:33,  1.22it/s]
1941it [25:34,  1.18it/s]
1942it [25:35,  1.13it/s]
1943it [25:36,  1.12it/s]
1943it [25:37,  1.26it/s]
Traceback (most recent call last):
  File "tools/demo.py", line 329, in <module>
    detect(cfg,opt)
  File "tools/demo.py", line 297, in detect
    with open(new_csv_path, "w", newline='') as new_csv:
PermissionError: [Errno 13] Permission denied: '../csv_raw_output\\20200306_123926_EF.mp4.csv'


## Closest boxes

In [ ]:
import os 
import csv

# create folder
if not (os.path.exists(closest_dir) and os.path.isdir(closest_dir)):
  os.mkdir(closest_dir)

for detection_csv in os.listdir(processed_csv_dir):
  detection_csv_path = os.path.join(processed_csv_dir, detection_csv)
  csv_array = []

  with open(detection_csv_path, "r") as detection_csv_file:
    csv_reader = csv.reader(detection_csv_file)
    csv_array = list(csv_reader)

  closest_array = []
  for detections in csv_array:
    frame_number = detections[0]
    left_car = find_largest_box(detections[1:], -1)
    zero_car = find_largest_box(detections[1:], 0)
    right_car = find_largest_box(detections[1:], 1)
    closest_array.append([frame_number, left_car, zero_car, right_car])


  # write to csv file
  closest_csv_dir = os.path.join(closest_dir, detection_csv)
  with open(closest_csv_dir, 'w', newline='') as closest_csv:
    csv_writer = csv.writer(closest_csv)
    csv_writer.writerows(closest_array)

## Generate video with bounding boxes

In [ ]:
import cv2
import os
import csv
import numpy as np
import math
import random

# either processed_csv_dir, or closest_dir
target_dir = closest_dir

print(os.listdir(csv_dir))
print(os.listdir(source_dir))
print(os.listdir(save_dir))

tl = 2
tf = max(tl - 1, 1)

for detection_csv in os.listdir(csv_dir):

  randomf = random.random()
  if randomf > 0.4:
    continue



  write_video_res = (1280, 720)
  detection_csv_path = os.path.join(target_dir, detection_csv)
  
  video_name = detection_csv.split(".csv")[0]

  source_video_path = os.path.join(source_dir, video_name)

  video_reader = cv2.VideoCapture(source_video_path)
  video_res = [int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))]
  #video_reader.release()


  detections_by_frame = {}
  csv_array = []

  

  with open(detection_csv_path, "r") as detection_csv_file:
    csv_reader = csv.reader(detection_csv_file)
    csv_array = list(csv_reader)
  
  video_output_path = os.path.join(save_dir, detection_csv.split(".csv")[0])

  fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
  video_writer = cv2.VideoWriter(video_output_path, fourcc, int(detection_fps), (write_video_res[0], write_video_res[1]))
  
  lane_number = None
  past_row = None

  for i in range(math.ceil(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))):

    if len(csv_array) > 0 and len(csv_array[0]) > 1 and int(float(csv_array[0][0])) == i:
      #print("passes")
      row = csv_array.pop(0)
      frame_number = int(float(row[0]))
    else:
      row = []
      frame_number = -1

    if detection_fps != int(math.ceil(video_reader.get(cv2.CAP_PROP_FPS))):
      print("not")
      video_reader.set(cv2.CAP_PROP_POS_FRAMES, i)

    success, frame = video_reader.read()
    if success:
      #print(frame_number)
      
      frame = cv2.resize(frame, write_video_res)
      if len(row) > 1:
        for detection in row[1:]:
          if detection is None or len(detection) <= 0:
            continue

          past_row = row
          print(detection)
          detection_data = read_csv_detection(detection)
          lane_number = None
          lane_number, x1, y1, x2, y2, conf, frame_center = detection_data[0:7]
          class_id = None
          if len(detection_data) > 7:
            class_id = detection_data[7]
          c1, c2 = (x1, y1), (x2, y2)
          if not lane_number is None:
            if lane_number == 0:
              color = [0, 255, 0]
            else:
              color = [0, 0, 255]
            cv2.rectangle(frame, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
                    
            label = str(lane_number)
            if(class_id == 3):
              label += " car"
            elif(class_id == 4):
              label += " motorcycle"
            elif(class_id == 6):
              label += " bus"
            elif(class_id == 8):
              label += " truck"
            t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
            c2 = c1[0] + t_size[0], c1[1] - t_size[1]
            cv2.rectangle(frame, c1, c2, color, -1, cv2.LINE_AA)  # filled
            cv2.putText(frame, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
      elif past_row and False == True:
        for detection in past_row[1:]:
          detection_data = read_csv_detection(detection)
          lane_number = None
          lane_number, x1, y1, x2, y2, conf, frame_center = detection_data[0:7]
          c1, c2 = (x1, y1), (x2, y2)
          if not lane_number is None:
            if lane_number == 0:
              color = [0, 255, 0]
            else:
              color = [0, 0, 255]
            cv2.rectangle(frame, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
        
            


      label2 = str(i)
      t_size2 = cv2.getTextSize(label2, 0, fontScale=2 / 3, thickness=1)[0]
      cv2.rectangle(frame, (0,0), (t_size2[0], t_size2[1]), (0,0,0), -1, cv2.LINE_AA)
      cv2.putText(frame, label2, (0, t_size2[1] + 3), 0, 2 / 3, [225, 255, 255], thickness=1, lineType=cv2.LINE_AA)

      video_writer.write(frame)
  video_writer.release()


## Test center

In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

test_center = 615
center_distance = 50
frame_number = 3000

video_reader = cv2.VideoCapture(os.path.join(source_dir, os.listdir(source_dir)[13]))
video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
video_res = [int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))]
print(video_res)

print(os.path.join(source_dir, os.listdir(source_dir)[4]))

success, frame = video_reader.read()
if success: 
  cv2.line(frame, (int(test_center * video_res[0] / 1280), int(video_res[1] * 1 / 3)), (int(test_center * video_res[0] / 1280), int(video_res[1] * 2 / 3)), (255,0,0), 3)
  cv2.line(frame, (int(test_center * video_res[0] / 1280) + center_distance, int(video_res[1] * 1 / 3)), (int(test_center * video_res[0] / 1280) + center_distance, int(video_res[1] * 2 / 3)), (0, 255,0), 3)
  cv2.line(frame, (int(test_center * video_res[0] / 1280) - center_distance, int(video_res[1] * 1 / 3)), (int(test_center * video_res[0] / 1280) - center_distance, int(video_res[1] * 2 / 3)), (0, 255,0), 3)
  #cv2.imwrite("../test.png", frame)
  cv2_imshow(frame)